## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

In [2]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json() 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [4]:
len(data)

843

In [5]:
import pandas as pd
df = pd.DataFrame(data)
df.head(1)


station_id                           external_id                     name  \
0        298  66db69d0-0aca-11e7-82f6-3863bb44ef7c  3 Ave & Schermerhorn St   

  short_name        lat        lon  region_id     rental_methods  capacity  \
0    4437.01  40.686832 -73.979677         71  [CREDITCARD, KEY]        35   

                                          rental_url  \
0  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   

   electric_bike_surcharge_waiver  eightd_has_key_dispenser  \
0                           False                     False   

                             eightd_station_services  has_kiosk  
0  [{'id': '5387339b-dcb0-4070-8e51-4fb719d3d482'...       True

In [6]:
# We drop the 'rental methods' column, as it contains multiple values and 
# we cannot insert lists in a database cell.
# also drop eightd_station_services for the same reason
df.drop('rental_methods', axis=1, inplace = True)
df.drop('eightd_station_services', axis=1, inplace = True)
df.head(5)

station_id                           external_id  \
0        298  66db69d0-0aca-11e7-82f6-3863bb44ef7c   
1        301  66db6aae-0aca-11e7-82f6-3863bb44ef7c   
2        337  66db9925-0aca-11e7-82f6-3863bb44ef7c   
3        358  66dbc8f5-0aca-11e7-82f6-3863bb44ef7c   
4        426  66dc0e99-0aca-11e7-82f6-3863bb44ef7c   

                            name short_name        lat        lon  region_id  \
0        3 Ave & Schermerhorn St    4437.01  40.686832 -73.979677         71   
1              E 2 St & Avenue B    5515.02  40.722174 -73.983688         71   
2            Old Slip & Front St    4993.12  40.703799 -74.008387         71   
3  Christopher St & Greenwich St    5847.01  40.732916 -74.007114         71   
4          West St & Chambers St    5329.03  40.717548 -74.013221         71   

   capacity                                         rental_url  \
0        35  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
1        58  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
2        37  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
3        50  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
4        29  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   

   electric_bike_surcharge_waiver  eightd_has_key_dispenser  has_kiosk  
0                           False                     False       True  
1                           False                     False       True  
2                           False                     False       True  
3                           False                     False       True  
4                           False                      True       True

### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [7]:
!sudo -H pip3 install -U sqlalchemy

sudo: effective uid is not 0, is /usr/bin/sudo on a file system with the 'nosuid' option set or an NFS file system without root privileges?


In [8]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = 'bigdata.stern.nyu.edu', 
    user = 'DealingF19',
    password = 'DealingF19!!')

engine = create_engine(conn_string)
con = engine.connect()

Once we have connected successfully, we need to create our database:

In [9]:
# Query to create a database
db_name = 'citibike_stations'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
engine.execute(create_db_query)
# And lets switch to the database
engine.execute("USE {db}".format(db=db_name))

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:552: Warning: (1007, "Can't create database 'citibike_stations'; database exists")
  cursor.execute(statement, parameters)


Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [10]:
# Drop the table Stations, just to be sure
drop_table_query= "drop  table Stations "
engine.execute(drop_table_query)

In [11]:
# This step is typically optional, but it is good practice to define explicitly the 
# data types before storing things in a database. In many cases, this can be ommitted, though.
import sqlalchemy
dtype = {
    'capacity': sqlalchemy.types.INT,
    'eightd_has_key_dispenser':  sqlalchemy.types.BOOLEAN,
    'lat': sqlalchemy.types.Float, 
    'lon': sqlalchemy.types.Float,
    'name': sqlalchemy.types.VARCHAR(50),
    'region_id': sqlalchemy.types.VARCHAR(5),
    'rental_url': sqlalchemy.types.VARCHAR(100),
    'short_name': sqlalchemy.types.VARCHAR(10),
    'station_id': sqlalchemy.types.INT
}

In [12]:
df.head(3)

station_id                           external_id                     name  \
0        298  66db69d0-0aca-11e7-82f6-3863bb44ef7c  3 Ave & Schermerhorn St   
1        301  66db6aae-0aca-11e7-82f6-3863bb44ef7c        E 2 St & Avenue B   
2        337  66db9925-0aca-11e7-82f6-3863bb44ef7c      Old Slip & Front St   

  short_name        lat        lon  region_id  capacity  \
0    4437.01  40.686832 -73.979677         71        35   
1    5515.02  40.722174 -73.983688         71        58   
2    4993.12  40.703799 -74.008387         71        37   

                                          rental_url  \
0  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
1  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
2  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   

   electric_bike_surcharge_waiver  eightd_has_key_dispenser  has_kiosk  
0                           False                     False       True  
1                           False                     False       True  
2                           False                     False       True

#### The to_sql method gives an error, but actually creates the table

In [13]:
table_name = 'Stations'



df.head(5)

# Create a table (this gives an error, but actually creates the Stations table)
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
df.to_sql(table_name, engine,  index = False, if_exists='replace', dtype=dtype)
    


AttributeError: 'NoneType' object has no attribute 'replace'

In [14]:
df.head(5)

station_id                           external_id  \
0        298  66db69d0-0aca-11e7-82f6-3863bb44ef7c   
1        301  66db6aae-0aca-11e7-82f6-3863bb44ef7c   
2        337  66db9925-0aca-11e7-82f6-3863bb44ef7c   
3        358  66dbc8f5-0aca-11e7-82f6-3863bb44ef7c   
4        426  66dc0e99-0aca-11e7-82f6-3863bb44ef7c   

                            name short_name        lat        lon  region_id  \
0        3 Ave & Schermerhorn St    4437.01  40.686832 -73.979677         71   
1              E 2 St & Avenue B    5515.02  40.722174 -73.983688         71   
2            Old Slip & Front St    4993.12  40.703799 -74.008387         71   
3  Christopher St & Greenwich St    5847.01  40.732916 -74.007114         71   
4          West St & Chambers St    5329.03  40.717548 -74.013221         71   

   capacity                                         rental_url  \
0        35  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
1        58  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
2        37  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
3        50  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
4        29  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   

   electric_bike_surcharge_waiver  eightd_has_key_dispenser  has_kiosk  
0                           False                     False       True  
1                           False                     False       True  
2                           False                     False       True  
3                           False                     False       True  
4                           False                      True       True

In [15]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = 'ALTER TABLE Stations ADD PRIMARY KEY(station_id)'
engine.execute(add_key_query)

### Reading from a SQL Database in Python using Pandas

We can similarly read from the database using Pandas

In [16]:
query =  "SELECT * FROM Stations"
df2 = pd.read_sql(query, con=engine)
df2.head(5)


station_id                           external_id  \
0          72  66db237e-0aca-11e7-82f6-3863bb44ef7c   
1          79  66db269c-0aca-11e7-82f6-3863bb44ef7c   
2          82  66db277a-0aca-11e7-82f6-3863bb44ef7c   
3          83  66db281e-0aca-11e7-82f6-3863bb44ef7c   
4         116  66db28b5-0aca-11e7-82f6-3863bb44ef7c   

                            name short_name      lat      lon region_id  \
0               W 52 St & 11 Ave    6926.01  40.7673 -73.9939        71   
1       Franklin St & W Broadway    5430.08  40.7191 -74.0067        71   
2         St James Pl & Pearl St    5167.06  40.7112 -74.0002        71   
3  Atlantic Ave & Fort Greene Pl    4354.07  40.6838 -73.9763        71   
4                W 17 St & 8 Ave    6148.02  40.7418 -74.0015        71   

   capacity                                         rental_url  \
0        55  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
1        33  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
2        27  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
3        62  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   
4        39  http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...   

   electric_bike_surcharge_waiver  eightd_has_key_dispenser  has_kiosk  
0                               0                         0          1  
1                               0                         0          1  
2                               0                         0          1  
3                               0                         0          1  
4                               0                         0          1

And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [17]:
# The necessary library to write in Excel
!pip3 install -U xlwt

     |████████████████████████████████| 102kB 19.4MB/s ta 0:00:01
ERROR: Could not install packages due to an EnvironmentError: [Errno 30] Read-only file system: '/usr/lib/python3.6/site-packages'

You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
df2.to_excel("citibike.xls")
df2.to_csv("citibike.csv")
!ls

ModuleNotFoundError: No module named 'xlwt'

### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.